In [17]:
import Pkg; 
Pkg.add("MathOptInterface")
include("CayleyVerify.jl")
include("DeepPoly.jl")

# utility functions
function dorefa_to_staircase(k::Int)
    n = 2^k - 1
    slopes = zeros(n+1)
    breakpoints = [-Inf]
    for i in 1:n
        push!(breakpoints, (2*i-1)/n - 1)
    end
    push!(breakpoints, Inf)
    
    constant_terms = [-1.0]
    for i in 1:n
        push!(constant_terms, -1.0 + 2*i/n)
    end
    return StaircaseFunction(breakpoints, slopes, constant_terms)
end

function predict(neural_net, img)
    num_layers = length(neural_net.weights)
    a = img'
    for i in 1:num_layers
        a = a * neural_net.weights[i] + neural_net.biases[i]'
        if i <= num_layers -1
            a = [eval(neural_net.activation[i], a[j]) for j in 1:length(a)]'
            print(a)
            print("\n----\n")
        end
    end
    output = a'
    print(output)
    return findmax(output)[2]
end

   Resolving package versions...
  No Changes to `~/OR-Research/LinSepVerify/Project.toml`
  No Changes to `~/OR-Research/LinSepVerify/Manifest.toml`


predict (generic function with 1 method)

In [18]:
import Pickle
using Suppressor
net_from_pickle = Pickle.load(open("./models/MNIST-DoReFa2_Dense256-Dense256.pkl"))
f = dorefa_to_staircase(2)
activation = [f, f]
neural_net = NeuralNetwork(net_from_pickle, activation)
print("net loaded")

net loaded

In [19]:
raw_imgs = Pickle.load(open("./imgs/MNIST_images-for-verification"))
imgs = []
for img in raw_imgs
    img = vcat([w' for w in img] ...)
    img = vcat(img'...)
    push!(imgs, img)
end
labels = Pickle.load(open("./imgs/MNIST_labels-for-verification"))
labels = Array{Int64}(labels.args[2][5])
labels = [l+1 for l in labels]
print("images loaded")

images loaded

# Big-M Formulation

In [20]:
using Suppressor
eps = 0.008
robust = 150
for (i,img) in enumerate(imgs)
    @suppress begin
        mip, variable_neuron_dict, neuron_integervar_dict = init_mip_deeppoly(neural_net, img, eps)
        true_label = labels[i]

        for j in 1:10
            if j != true_label
                target_label = j
                last_layer = last(neural_net.weights)
                objective = zeros(10)
                objective[target_label] = 1.0
                objective[true_label] = -1.0
                c = last_layer * objective
                num_layers = length(neural_net.weights)
                final_dim, output_dim = size(last_layer)
                @objective(mip, Max, sum(c[i]*mip[:x][num_layers, i] for i in 1:final_dim))
                optimize!(mip)
                opt_val = objective_value(mip)
                if opt_val > 0
                    robust -= 1
                    break
                end
            end
        end
    end
end

In [ ]:
robust

# Cayley Embedding

In [ ]:
eps = 0.032
robust = 150
for (i,img) in enumerate(imgs)
    print("verifying image ", i)
    print("\n")
    @suppress begin
        true_label = labels[i]
        for j in 1:10
            if j != true_label
                target_label = j
                opt_val, opt_sol_x, opt_sol_z, mip = target_attack(neural_net, img, true_label, target_label, eps)
                if opt_val > 0
                    robust -= 1
                    break
                end
            end
        end
    end
end


In [ ]:
robust